In [1]:
from keras.optimizers import SGD
from convnetskeras.convnets import preprocess_image_batch, convnet
from convnetskeras.imagenet_tool import synset_to_dfs_ids
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tq
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import pickle
from keras.models import load_model
from keras.models import Sequential
from keras.layers import *
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing.image import load_img,img_to_array
import itertools
from convnetskeras.customlayers import Softmax4D
import gc

Using Theano backend.
Using gpu device 0: GeForce GTX 960M (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5105)


In [2]:
def layer_type(layer):
    return str(layer)[10:].split(" ")[0].split(".")[-1]

def detect_configuration(model):
    # must return the configuration and the number of the first pooling layer
    
    # Names (types) of layers from end to beggining
    inverted_list_layers = [layer_type(layer) for layer in model.layers[::-1]]
    
    layer1 = None
    layer2 = None 
    
    i = len(model.layers)
    
    for layer in inverted_list_layers:
        i -= 1
        if layer2 is None:
            if layer == "GlobalAveragePooling2D" or layer == "GlobalMaxPooling2D":
                layer2 = layer

            elif layer == "Flatten":
                return "local pooling - flatten", i
            
        else:
            layer1 = layer
            break
            
    if layer1 == "MaxPooling2D" and layer2 == "GlobalMaxPooling2D":
        return "local pooling - global pooling (same type)", i
    elif layer1 == "AveragePooling2D" and layer2 == "GlobalAveragePooling2D":
        return "local pooling - global pooling (same type)", i
    
    elif layer1 == "MaxPooling2D" and layer2 == "GlobalAveragePooling2D":
        return "local pooling - global pooling (different type)", i+1
    elif layer1 == "AveragePooling2D" and layer2 == "GlobalMaxPooling2D":
        return "local pooling - global pooling (different type)", i+1
    
    else:
        return "global pooling", i
    
def insert_weights(layer, new_layer):
    W,b = layer.get_weights()
    print(W.shape)
    n_filter,previous_filter,ax1,ax2 = new_layer.get_weights()[0].shape
    new_W = W.reshape((previous_filter,ax1,ax2,n_filter))
    new_W = new_W.transpose((3,0,1,2))
    new_W = new_W[:,:,::-1,::-1]
    new_layer.set_weights([new_W,b])
    
def copy_last_layers(model, new_model, begin):
    
    i=begin
    
    for layer in model.layers[begin:]:
        if layer_type(layer) == "Dense":
                
            add_reshaped_layer(layer, new_model, 1)
                
        elif layer_type(layer) == "Activation" and i == len(model.layers)-1:
            break
               
        else:
            print(i)
            print(len(model.layers))
            print(layer_type(layer))
            new_model.add(layer)
        i+=1
    
    model.add(Softmax4D(axis=1,name="softmax"))
    
                
def add_reshaped_layer(layer, new_model, size):

    conf = layer.get_config()
    
    new_layer = Convolution2D(conf["output_dim"],size,size,activation=conf["activation"], name=conf['name'])
         
        
    new_model.add(new_layer)
    # We transfer the weights:
    insert_weights(layer, new_layer)
    

def to_heatmap_sequential(model, size=None, increased_resolution=False, sequential=False, delete = False):
    
    # First, we need to find which layers need special attention. 
    # We do that by examining the last layers and stop once we see a pool layer. (what if 2 pooling layers?)
    # eg: averagePool -> globalPool?
    # This is for later. Right now, we need a simple heatmap.
    
    # there are four configurations possible:
    # global pooling
    # local pooling - flatten
    # local pooling - global pooling (same type)
    # local pooling - global pooling (different type)
    
    model_type, index = detect_configuration(model)
    
    # We reproduce the beggining of the network.
    new_model = Sequential()
    for layer in model.layers[:index]:
        new_model.add(layer)
        
    if model_type == "global pooling":
        copy_last_layers(model, new_model, index+1)
              
    elif model_type == "local pooling - flatten":
        
        add_reshaped_layer(model.layers[index+1], new_model, size)
        
        copy_last_layers(model, new_model, index+2)
        
        
    elif model_type == "local pooling - global pooling (same type)":
        copy_last_layers(model, new_model, index+2)
    elif model_type == "local pooling - global pooling (different type)":
        copy_last_layers(model, new_model, index+1)
    else:
        raise IndexError("no type for model: " + str(model_type))
    
    if delete:
        del(model)
        gc.collect()
    
    return new_model


In [3]:
25088/512

49.0

In [4]:
new_model = to_heatmap_sequential(model, 7)

NameError: name 'model' is not defined

In [ ]:
model=new_model

In [ ]:
model.layers[-4].get_config()

In [ ]:
str(model.layers[-1])[10:].split(" ")[0].split(".")[-1]

In [ ]:
model.layers[-31].get_weights()[0].shape

In [ ]:
print(model.summary())

In [ ]:
def to_heatmap_functional(model, resolution_x2=False, sequential=False):
    

In [ ]:
def pop_layer(model):
    if not model.outputs:
        raise Exception('Sequential model cannot be popped: model is empty.')

    model.layers.pop()
    if not model.layers:
        model.outputs = []
        model.inbound_nodes = []
        model.outbound_nodes = []
    else:
        model.layers[-1].outbound_nodes = []
        model.outputs = [model.layers[-1].output]
    model.built = False

In [ ]:
import os
from fnmatch import fnmatch

l = []

root = './train/'
pattern = "*.jpg"

for path, subdirs, files in os.walk(root):
    for name in files:
        if fnmatch(name, pattern):
            l.append(os.path.join(path, name))

In [ ]:
l[0]

In [ ]:
s = 7
dest = [ string[:s] + "_heatmap" + string[s:] for string in l]
dest[0]

In [ ]:
dest_pickle = [ string[:s] + "_pickle" + string[s:] for string in l]

In [ ]:
def get_heatmap(path,model):
        
    im = preprocess_image_batch([path], color_mode="bgr")

    out = model.predict(im)

    s = "n02512053"
    ids = synset_to_dfs_ids(s)
    heatmap = out[0,ids].sum(axis=0)
    return heatmap


def save_heatmaps(model,l,dest_pickle, dest):
    for i in tq(range(len(l))):

        heatmap = get_heatmap(l[i], model)

        im = preprocess_image_batch([l[i]], color_mode="bgr")

        plt.imsave(dest[i],heatmap)
        
        with open(dest_pickle[i],"wb") as f:
            pickle.dump(heatmap, f)
        

In [8]:
pb = tq(total=47)
old_model = convnet('vgg_19',weights_path="vgg19_weights.h5", heatmap=True)
pb.update()
model = Sequential()

model.add(ZeroPadding2D((1,1),input_shape=(3,None,None)))
model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_4'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_4'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
    
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_4'))
model.add(MaxPooling2D((2,2), strides=(1,1)))

model.add(Convolution2D(4096,14,14,activation="relu",name="custom_dense_1"))
model.add(Convolution2D(4096,1,1,activation="relu",name="dense_2"))
model.add(Convolution2D(1000,1,1,name="dense_3"))
model.add(Softmax4D(axis=1,name="softmax"))

pb.update()

for layer in model.layers:
    
    if layer.name.startswith("custom_"):
        orig_layer = old_model.get_layer(layer.name[7:])
        
        with open("big_matrix.p", "rb") as f:
            new_weights = pickle.load(f)
        
        layer.set_weights([new_weights, orig_layer.get_weights()[1]])
    elif layer.name.startswith("conv") or layer.name.startswith("dense"):
        orig_layer = old_model.get_layer(layer.name)
        layer.set_weights(orig_layer.get_weights())
    pb.update()

#model.compile(optimizer='sgd', loss='mse')
pb.close()

MemoryError: ('Error allocating 1644167168 bytes of device memory (CNMEM_STATUS_OUT_OF_MEMORY).', "you might consider using 'theano.shared(..., borrow=True)'")

In [ ]:
gc.collect()

In [ ]:
print(model.summary())

In [ ]:
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model = convnet('vgg_19',weights_path="vgg19_weights.h5", heatmap=False)
model.compile(optimizer=sgd, loss='mse')

In [ ]:
str(type(model.layers[4]))

In [ ]:
with open("big_matrix.p", "rb") as f:
    new_weights = pickle.load(f)

In [ ]:
model.layers[-4].get_weights()[0][0][0]

In [ ]:
im = preprocess_image_batch(["./train/ALB/img_00110.jpg"], color_mode="bgr")

out = model.predict(im)

s = "n02512053"
ids = synset_to_dfs_ids(s)
heatmap = out[0,ids].sum(axis=0)

In [ ]:
plt.imshow(heatmap, interpolation="none")
plt.show()

In [ ]:
im = preprocess_image_batch(["./train/ALB/img_00136.jpg"], color_mode="bgr")

out = model.predict(im)

s = "n02512053"
ids = synset_to_dfs_ids(s)
heatmap = out[0,ids].sum(axis=0)
plt.imshow(heatmap, interpolation="none")
plt.show()

In [ ]:
print(model.summary())

In [9]:
my_weights = model.layers[-4].get_weights()

In [10]:
new = np.zeros(my_weights[0].shape[:-2] + (14,14))

In [12]:
new.shape

(512, 512, 14, 14)

In [13]:
type(my_weights[0])

numpy.ndarray

In [15]:
c = tq(total=4096*512*14*14)
for i, j, k, l in itertools.product(range(new.shape[0]),range(new.shape[1]),range(new.shape[2]),range(new.shape[3])):
    if k%2 == 0 or l%2 == 0:
        new[i,j,k,l] = 0
    else:
        new[i,j,k,l] = my_weights[0][i,j, int((k-1)/2), int((l-1)/2)]
    c.update()
c.close()

IndexError: index 3 is out of bounds for axis 3 with size 3

In [ ]:
with open("big_matrix_2.p", "wb") as f:
    pickle.dump(new, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
#save_heatmaps(model,l,dest_pickle)

In [16]:
a = get_heatmap(l[0], model)

NameError: name 'get_heatmap' is not defined

In [ ]:
threshold = 0.3

def binarize(value, threshold):
    if value < threshold:
        return 0
    if value >= threshold:
        return 1
np_binarize = np.vectorize(binarize)

In [ ]:
a

In [ ]:
b = np_binarize(a, threshold)

In [ ]:
plt.imshow(b, interpolation='none')
plt.show()

In [ ]:
def change_class(value, old, new):
    if value == old:
        return new
    else:
        return value
np_change_class = np.vectorize(change_class)




def stuff(mat,i,j,islands_list, count, islands_matrix):
    if mat[i][j] == 0:
        pass
    else:
        before = int(islands_matrix[i][j-1])
        above = int(islands_matrix[i-1][j])
        if before == 0:
            if  above== 0:
                islands_matrix[i][j] = count
                count +=1
                islands_list.append([(j,i)])
            else: 
                islands_matrix[i][j] = above
                islands_list[above-1].append((j,i))
        
        else:
            if above == 0:
                islands_matrix[i][j] = before
                islands_list[before-1].append((j,i))
            
            
            else:
                if above == before:
                    islands_matrix[i][j] = before
                    islands_list[before-1].append((j,i))
                
                #it's on.
                else:
                    old = max((above,before))
                    new = min((above,before))
                    
                    islands_matrix[i][j] = new
                    islands_matrix = np_change_class(islands_matrix, old, new)
                    islands_list[new-1] += islands_list[old-1]
                    islands_list[new-1].append((j,i))
                    islands_list[old-1] = []
                    
    return islands_list, count, islands_matrix        


# The not fun part
def group_by_island(matrix):
    
    h, l = np.shape(matrix)
    border = np.array([[0]*(l)])
    mat = np.concatenate((border,matrix), axis=0)
    border = np.array([[0]*(h+1)])
    mat = np.concatenate((border.T, mat), axis=1)
    
    islands_matrix =np.zeros((h+1,l+1))
    count = 1
    islands_list = []
    
    for i in range(1,h+1):
        for j in range(1, l+1):
            islands_list, count, islands_matrix = stuff(mat,i,j,islands_list, count, islands_matrix)

    
    
    return islands_matrix, islands_list
    

In [ ]:
islands_matrix, islands_list = group_by_island(b)
plt.imshow(islands_matrix, interpolation='none')
plt.show()

In [ ]:
def find_extremes(list_coordinates):
    max_x = max(list_coordinates, key = lambda x: x[0])[0]
    min_x = min(list_coordinates, key = lambda x: x[0])[0]
    max_y = max(list_coordinates, key = lambda x: x[1])[1]
    min_y = min(list_coordinates, key = lambda x: x[1])[1]
    
    #print(min_x, min_y, max_x, max_y )
    
    return min_x - 1, min_y - 1, max_x-min_x + 1, max_y-min_y + 1


def get_rectangles(islands_list, area_limit=4):
    list_coordinates = []
    for island in islands_list:
        if len(island) == 0:
            continue
        else:
            coordinates = find_extremes(island)
            #print(coordinates)
            #print(coordinates[2]*coordinates[3])
            if coordinates[2]*coordinates[3] >=area_limit:
                #print(coordinates)
                list_coordinates.append(coordinates)
            
    return list_coordinates
    

In [ ]:
rectangles = get_rectangles(islands_list)

In [ ]:
print(rectangles[0])

In [ ]:
def convert_coordinates(rectangles, heatmap, path_image):
    y, x = np.shape(heatmap)
    #print('y=' + str(y))
    #print("x=" + str(x))
    
    img = np.array(Image.open(path_image))
    y1, x1, channel = np.shape(img)
    #print("x1=" + str(x1))
    #print("y1=" + str(y1))
    
    ratio_x = x1/x
    ratio_y = y1/y
    
    converted_rectangles = []
    for r in rectangles:
        converted_rectangles.append((r[0]*ratio_x, r[1]*ratio_y, r[2]*ratio_x, r[3]*ratio_y,))
        
    return converted_rectangles

In [ ]:
new_rectangles = convert_coordinates(rectangles, a, l[0])

In [ ]:
new_dic = {"filename": l[0], "rectangles": new_rectangles}

In [ ]:
new_dic

In [ ]:
c=0.4
def from_pickle_to_rectangle(img_path, pickle_path):
    
    with open(pickle_path,"rb") as f:
        heatmap = pickle.load(f)
        
    # Here we will modify the heatmap, removing activation on the extremes of the image, because
    # cause the fish detector to activate too often for some reason.
    line_zeros = np.zeros(np.shape(heatmap)[0])
    column_zeros = np.zeros(np.shape(heatmap)[1])
    heatmap[:,0] = line_zeros
    heatmap[:,-1] = line_zeros
    heatmap[0,] = column_zeros
    heatmap[-1,:] = column_zeros
    
        
    #print(np.shape(heatmap))
    threshold = c*np.max(heatmap)
    binary_map = np_binarize(heatmap, threshold)
    islands_matrix, islands_list = group_by_island(binary_map)
    rectangles = get_rectangles(islands_list, area_limit=4)
    #print(rectangles[0])
    new_rectangles = convert_coordinates(rectangles, heatmap, img_path)
    #print(new_rectangles[0])
    new_dic = {"path": img_path, "rectangles": new_rectangles, "p_fish": np.max(heatmap)}
    
    
    return new_dic

In [ ]:
list_rectangles_images = []
for i in tq(range(len(l))):
    dic = from_pickle_to_rectangle(l[i], dest_pickle[i])
    list_rectangles_images.append(dic)

In [ ]:
import os
from fnmatch import fnmatch

l = []

root = './test_stg1/'
pattern = "*.jpg"

for path, subdirs, files in os.walk(root):
    for name in files:
        if fnmatch(name, pattern):
            l.append(os.path.join(path, name))

In [ ]:
l[0]

In [ ]:
s = 11
dest = [ string[:s] + "_heatmap" + string[s:] for string in l]
dest[0]

In [ ]:
dest_pickle = [string[:s] + "_pickle" + string[s:] for string in l]

In [ ]:
#save_heatmaps(model,l,dest_pickle, dest)

In [ ]:
list_rectangles_images = []
for i in tq(range(len(l))):
    dic = from_pickle_to_rectangle(l[i], dest_pickle[i])
    list_rectangles_images.append(dic)

In [ ]:
with open("test_rectangles.p", "wb") as f:
    pickle.dump(list_rectangles_images,f)

In [ ]:
my_list[0:5]

In [ ]:
for i in range(50,70):
    dic = list_rectangles_images[i]
    img = Image.open(dic["path"])
    print(str(dic["p_fish"]*100) + "        " + dic["path"])
    plt.imshow(img)
    plt.show()
    
    with open(dest_pickle[i], "rb") as f:
        heatmap = pickle.load(f)
        
    
    line_zeros = np.zeros(np.shape(heatmap)[0])
    column_zeros = np.zeros(np.shape(heatmap)[1])
    heatmap[:,0] = line_zeros
    heatmap[:,-1] = line_zeros
    heatmap[0,] = column_zeros
    heatmap[-1,:] = column_zeros
        
    plt.imshow(heatmap, interpolation='none')
    plt.show()
    
    threshold = c*np.max(heatmap)
    binary_map = np_binarize(heatmap, threshold)
    
    plt.imshow(binary_map, interpolation='none')
    plt.show()
    
    for rec in dic["rectangles"]:
        print("fish found: " + str(rec))
        img2 = img.crop((rec[0],rec[1], rec[0] + rec[2], rec[1] + rec[3]))
        plt.imshow(img2)
        plt.show()

In [ ]:
def get_sea_rectangle(predict):
    
    
    
    # We make the assumpption that the standart image size is 57x101
    # the order is x top left, y top left, x bottom right, y bottom right
    # This is the string order, not the numerical order! beware!
    if predict ==1:
        rectangle = [[58,0,100,56]]
    if predict ==10:
        rectangle = [[0,45,15,56]]
    if predict ==11:
        rectangle = [[80,0,100,56],[0,0,20,20]]
    if predict ==12:
        rectangle = [[0,0,20,56]]
    if predict ==13:
        rectangle = [[0,30,100,56]]
    if predict ==14:
        rectangle = [[85,0,100,56],[0,40,30,56]]
    if predict ==15:
        rectangle = [[0,0,100,15],[0,0,30,30]]
    if predict ==16:
        rectangle = [[0,0,10,56]]
    if predict ==17:
        rectangle = [[55,0,100,56]]
    if predict ==2:
        rectangle = [[50,0,100,56]]
    if predict ==3:
        rectangle = [[0,0,25,56]]
    if predict ==4:
        rectangle = [[0,0,100,15]]
    if predict ==5:
        rectangle = [[0,0,100,20],[65,0,100,56]]
    if predict ==6:
        rectangle = [[0,0,100,25],[0,0,20,56]]
    if predict ==7:
        rectangle = [[0,0,20,56],[75,0,100,20]]
    if predict ==8:
        rectangle = [[0,0,20,56]]
    if predict ==9:
        rectangle = [[80,0,100,56]]
    return rectangle
    

In [ ]:
del model

In [ ]:
model = load_model("cool_model_boats_id.h5")

In [ ]:
def remove_patches(heatmap, rectangles):
    y_map, x_map = np.shape(heatmap)
    for rectangle in rectangles: 
        x_min = int(rectangle[0] *x_map/100)
        x_max = int(rectangle[2] *x_map/100)
        y_min = int(rectangle[1] *y_map/56)
        y_max = int(rectangle[3] *y_map/56)
        
        for i in range(y_map):
            for j in range(x_map):
                if y_min<=i<y_max and x_min<=j<x_max:
                    heatmap[i,j] = 0
                    
    return heatmap

In [ ]:
def hide_sea(heatmap, img_path, treashold = 0.87):
    img = load_img(img_path,grayscale=True, target_size=(64,64))
    img = img_to_array(img)
    
    probabilities = model.predict(np.array([img,]))[0][1:]
    #print(probabilities)
    
    if np.max(probabilities) >= treashold:
        rectangles = get_sea_rectangle(np.argmax(probabilities) + 1)
        heatmap = remove_patches(heatmap, rectangles)
    
    return heatmap

In [ ]:
c=0.4
def from_pickle_to_rectangle(img_path, pickle_path, plot=False):
    
    with open(pickle_path,"rb") as f:
        heatmap = pickle.load(f)
        
    # Here we will modify the heatmap, removing activation on the extremes of the image, because
    # cause the fish detector to activate too often for some reason.
    line_zeros = np.zeros(np.shape(heatmap)[0])
    column_zeros = np.zeros(np.shape(heatmap)[1])
    heatmap[:,0] = line_zeros
    heatmap[:,-1] = line_zeros
    heatmap[0,] = column_zeros
    heatmap[-1,:] = column_zeros
    
    if plot:
        img = Image.open(img_path)
        plt.imshow(img)
        plt.show()
        plt.imshow(heatmap, interpolation='none')
        plt.show()
    
    heatmap = hide_sea(heatmap, img_path)
    
    if plot:
        plt.imshow(heatmap, interpolation='none')
        plt.show()
        
    #print(np.shape(heatmap))
    threshold = c*np.max(heatmap)
    binary_map = np_binarize(heatmap, threshold)
    
    if plot:
        plt.imshow(binary_map, interpolation='none')
        plt.show()
    
    islands_matrix, islands_list = group_by_island(binary_map)
    rectangles = get_rectangles(islands_list, area_limit=4)
    new_rectangles = convert_coordinates(rectangles, heatmap, img_path)
    new_dic = {"path": img_path, "rectangles": new_rectangles, "p_fish": np.max(heatmap)}
    
    if plot:
        for rec in new_dic["rectangles"]:
            print("fish found: " + str(rec))
            img2 = img.crop((rec[0],rec[1], rec[0] + rec[2], rec[1] + rec[3]))
            plt.imshow(img2)
            plt.show()
    
    return new_dic

In [ ]:
list_rectangles_images = []
for i in tq(range(50,70)):
    dic = from_pickle_to_rectangle(l[i], dest_pickle[i], True)
    list_rectangles_images.append(dic)